In [1]:
# Set up imports
import pandas as pd
import numpy as np
import scipy.stats as stats
import sqlalchemy as sql
import matplotlib.pyplot as plt

In [19]:
ieprawdf = pd.read_csv('res/2016-2017_Student_Discipline_Annual_Report_-_IEP.csv')

In [20]:
simplecols = ['dbn', 'locname', 'loccat', 'dist', 'genR', 'genP', 'genS', 'genE', 'swdR', 'swdP', 'swdS', 'swdE', 'total']
rawdf.columns = simplecols
# rawdf.head()

In [21]:
# normalization --> melt, merge
iepnormdf = rawdf.drop(['locname', 'loccat', 'dist'], axis=1)
iepnormdf = pd.melt(iepnormdf, id_vars=['dbn'])
iepnormdf.rename(columns={'value':'number'}, inplace=True)
        
demo_dict = {'g':'not swd', 't':'all students', 's':'swd'}
iepnormdf['var_name'] = [d[0] for d in iepnormdf['variable']]
iepnormdf['var_name'] = iepnormdf['var_name'].map(demo_dict)

iepnormdf['discipline_type'] = [d[-1] if d != 'total' else 'all' for d in iepnormdf['variable']]
iepnormdf['var_cat'] = ['disability' for v in iepnormdf['var_name']]
iepnormdf['var_type'] = ['demographic' for v in iepnormdf['var_name']]

iepnormdf = iepnormdf.drop('variable', axis=1)

iepnormdf = iepnormdf[['dbn', 'number', 'discipline_type', 'var_name', 'var_cat', 'var_type']]

print(iepnormdf.loc[iepnormdf['dbn'] == '25Q250'])
# print(len(iepnormdf))

          dbn number discipline_type      var_name     var_cat     var_type
1237   25Q250      R               R       not swd  disability  demographic
2902   25Q250     20               P       not swd  disability  demographic
4567   25Q250     10               S       not swd  disability  demographic
6232   25Q250      R               E       not swd  disability  demographic
7897   25Q250      R               R           swd  disability  demographic
9562   25Q250     22               P           swd  disability  demographic
11227  25Q250     22               S           swd  disability  demographic
12892  25Q250      R               E           swd  disability  demographic
14557  25Q250     91             all  all students  disability  demographic


In [ ]:
categories_dict = {'D_CODE': ['d-code', 'discipline'], \
                   'ELL': ['ell status', 'demographic'], \
                   'Gender': ['gender', 'demographic'], \
                   'IEP': ['disability', 'demographic'], \
                   'LENGTH_IN_DAYS': ['removal time', 'discipline'], \
                   'RACE': ['race', 'demographic'], \
                   'Students_In_Temporary_Housing': ['housing status', 'demographic']}

type_dict = {'ELL'}

def general_clean(file, category):
    rawdf = pd.read_csv('res/2016-2017_Student_Discipline_Annual_Report_-_' + file + '.csv')
    newcols = [c.lower() for c in rawdf.columns]
    rawdf.columns = newcols
    
    rawdf.drop(['location name', 'location category', 'administrative district', 'sy1617 total'], axis=1, inplace=True)
    
    normdf = pd.melt(rawdf, id_vars=['dbn'])
    
    normdf.rename(columns={'value':'number'}, inplace=True)
    
    normdf['variable'] = normdf['variable'].map(lambda x: x.split(' '))
    
    normdf['discipline_type'] = [v[-1] for v in normdf['variable']] 
    discipline_dict = {'removals':'r', 'principal':'p', 'superintendent': 's', 'expulsions':'e', 'removals/suspensions':'all'}
    normdf['discipline_type'] = normdf['discipline_type'].map(discipline_dict)
    
    normdf['var_name'] = [' '.join(v[:-1]) for v in normdf['variable']]
    
    normdf.drop('variable', axis=1, inplace=True)
    
    normdf['var_cat'] = [category[0] for i in normdf.index()]
    normdf['var_type'] = [category[1] for i in normdf.index()]

In [35]:
dict1 = {'a': 'z', 'b':'y', 'c':'x'}

for blah, meh in dict1.items():
    print(meh)
    print(type(meh))

z
<class 'str'>
y
<class 'str'>
x
<class 'str'>


In [ ]:
for file, category in categories_dict.items():
    general_clean(file, category)

In [22]:
racerawdf = pd.read_csv('res/2016-2017_Student_Discipline_Annual_Report_-_RACE.csv')
genderrawdf = pd.read_csv('res/2016-2017_Student_Discipline_Annual_Report_-_GENDER.csv')

racecols = [c.lower() for c in racerawdf.columns]
gendercols = [c.lower() for c in genderrawdf.columns]

racerawdf.columns = racecols
genderrawdf.columns = gendercols

racerawdf.drop(['location name', 'location category', 'administrative district'], axis=1, inplace=True)
genderrawdf.drop(['location name', 'location category', 'administrative district'], axis=1, inplace=True)

# print(rawdf.loc[rawdf['dbn'] == '25Q250'])

In [23]:
racedf = pd.melt(racerawdf, id_vars=['dbn'])
genderdf = pd.melt(genderrawdf, id_vars=['dbn'])

racedf['variable'] = racedf['variable'].map(lambda x: x.split(' '))
genderdf['variable'] = genderdf['variable'].map(lambda x: x.split(' '))

racedf.rename(columns={'value':'number'}, inplace=True)
genderdf.rename(columns={'value':'number'}, inplace=True)

racedf['discipline_type'] = [v[-1] for v in racedf['variable']]
genderdf['discipline_type'] = [v[-1] for v in genderdf['variable']]

discipline_dict = {'removals':'r', 'principal':'p', 'superintendent': 's', 'expulsions':'e', 'removals/suspensions':'all'}
racedf['discipline_type'] = racedf['discipline_type'].map(discipline_dict)
genderdf['discipline_type'] = genderdf['discipline_type'].map(discipline_dict)

racedf['var_name'] = [' '.join(v[:-1]) for v in racedf['variable']]#race terms with spaces ('American Indian', etc.)
genderdf['var_name'] = [' '.join(v[:-1]) for v in genderdf['variable']]

racedf['var_name'] = racedf['var_name'].map(lambda x: 'all students' if x=='sy1617 total' else x)
genderdf['var_name'] = genderdf['var_name'].map(lambda x: 'all students' if x=='sy1617 total' else x)

racedf['var_name'] = racedf['var_name'].map(lambda x: 'race unknown' if x=='unknown' else x)
# no unknown gender column

racedf.drop('variable', axis=1, inplace=True)
genderdf.drop('variable', axis=1, inplace=True)

racedf['var_cat'] = ['race' for v in racedf['var_name']]
genderdf['var_cat'] = ['gender' for v in genderdf['var_name']]

racedf['var_type'] = ['demographic' for v in racedf['var_name']]
genderdf['var_type'] = ['demographic' for v in genderdf['var_name']]

# print(racedf.loc[racedf['dbn'] == '25Q250'])
# print(len(racedf))

genderdf.head()

,dbn,number,discipline_type,var_name,var_cat,var_type
0,01M015,R,r,female,gender,demographic
1,01M019,R,r,female,gender,demographic
2,01M020,R,r,female,gender,demographic
3,01M034,R,r,female,gender,demographic
4,01M063,R,r,female,gender,demographic


In [25]:
suspensionsdf = iepnormdf.append([racedf, genderdf])


print('IEP: ' + str(len(iepnormdf)) + ' rows')
print('Race: ' + str(len(racedf)) + ' rows')
print('Gender: ' + str(len(genderdf)) + ' rows')
print('All suspensions: ' + str(len(suspensionsdf)) + ' rows')
print(suspensionsdf.loc[suspensionsdf['dbn'] == '05M285'])

IEP: 14985 rows
Race: 48285 rows
Gender: 14985 rows
All suspensions: 78255 rows
          dbn number discipline_type                        var_name  \
243    05M285      R               R                         not swd   
1908   05M285      R               P                         not swd   
3573   05M285      7               S                         not swd   
5238   05M285      R               E                         not swd   
6903   05M285      R               R                             swd   
8568   05M285      R               P                             swd   
10233  05M285      6               S                             swd   
11898  05M285      R               E                             swd   
13563  05M285     26             all                    all students   
244    05M285      R               r  american indian/alaskan native   
1909   05M285      R               p  american indian/alaskan native   
3574   05M285      R               s  american indian/al